# Calibration of Black and Scholes, Merton, Kou, Variance Gamma parameters
This notebook aims to find the optimal parameters of **Black-Scholes**, **Merton Jump Diffusion**, **Kou Jump Diffusion** and **Variance Gamma** models. To do so, we compute the european option prices using **closed formulas**, available for all the 4 models, and the **Fast Fourier Transform** for the VG model. Given these theoretical prices, the **implied volatilities** are computed comparing them with real market prices, minimizing their difference. Then we estimate the additional parameters of each model, using the python module `scipy.optimize`.

*reference: https://github.com/cantaro86/Financial-Models-Numerical-Methods/tree/master*


In [1]:
from functions.MERTONpricer import Merton_pricer
from functions.BSpricer import BS_Pricer
from functions.KOUpricer import Kou_pricer
from functions.VGpricer import VG_pricer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import scipy as scp
import scipy.stats as ss
import scipy.optimize as scpo
import time

random.seed(100)

Let's start retrieving historical prices for **european call** and **put** options starting from date **2016-01-20** and expiring **1 year** later. All the data have been collected from [OptionsDX](https://www.optionsdx.com/shop/) and preprocessed in `plainvanilla.py` module. Only options such that
$$ K = (1 \pm 0.2) * S0 $$ have been selected.

In [2]:
df_call = pd.read_csv('data/options_spx_call_OTM_2016.csv')
df_put = pd.read_csv('data/options_spx_put_OTM_2016.csv')
print(f'NUM OF CALL OPTS: {df_call.shape[0]}')
print(df_call)
print(f'NUM OF PUT OPTS: {df_put.shape[0]}')
print(df_put)

NUM OF CALL OPTS: 38
    QUOTE_DATE  UNDERLYING_LAST EXPIRE_DATE   C_BID   C_ASK  STRIKE     C_IV
0   2016-01-20          1859.48  2017-01-20  128.49  131.80  1875.0  0.20726
1   2016-01-20          1859.48  2017-01-20  115.50  118.60  1900.0  0.20243
2   2016-01-20          1859.48  2017-01-20  103.00  106.10  1925.0  0.19893
3   2016-01-20          1859.48  2017-01-20   91.20   94.29  1950.0  0.19460
4   2016-01-20          1859.48  2017-01-20   80.20   83.10  1975.0  0.18961
5   2016-01-20          1859.48  2017-01-20   70.00   72.91  2000.0  0.18595
6   2016-01-20          1859.48  2017-01-20   60.40   63.20  2025.0  0.18124
7   2016-01-20          1859.48  2017-01-20   51.40   54.10  2050.0  0.17713
8   2016-01-20          1859.48  2017-01-20   43.51   46.20  2075.0  0.17332
9   2016-01-20          1859.48  2017-01-20   36.19   38.70  2100.0  0.16809
10  2016-01-20          1859.48  2017-01-20   29.70   32.10  2125.0  0.16430
11  2016-01-20          1859.48  2017-01-20   24.11   2

The dataframes **df_calls** and **df_put** contain both 83 options sorted by strike price. For our purpose, using the $25\%$ of these DFs is enough and can lead to very good results. Thus, we sample the rows and then reformulate the dataframes adding **Midpoint** and **Spread** columns.

In [3]:
calls = df_call.sample(frac=0.2, replace=False).sort_index().reset_index(drop=True)
puts = df_put.sample(frac=0.2, replace=False).sort_index().reset_index(drop=True)

calls['C_Midpoint'] = abs(calls['C_BID'] + calls['C_ASK']) / 2
puts['P_Midpoint'] = abs(puts['P_BID'] + puts['P_ASK']) / 2
calls['C_Spread'] = calls['C_BID'] - calls['C_ASK']
puts['P_Spread'] = puts['P_BID'] - puts['P_ASK']

q = 0           # dividend yield
r = 0.03        # risk-free interest rate
sigma = 0.3     #volatility (variance of diffusion process)
S0 = calls.iloc[0]['UNDERLYING_LAST']
T = 1           # time-to-maturity
call_strikes = calls['STRIKE']    # array of K for call options
put_strikes = puts['STRIKE']      # array of K for put options
exercise = 'european'

call_prices = calls['C_Midpoint']
put_prices = puts['P_Midpoint']

The following code snippet initializes objects of class *BS_pricer*, *Merton_pricer*, *Kou_pricer*, *VG_pricer* with default values as parameters. Then it computes the theoretical call prices using closed formulas of each 4 models, with strike prices given by the *call_strikes* array. Additionally, we use the **Midpoint** price as the option market prices.P

In [4]:
BS = BS_Pricer(S0=S0, r=r, q = q, sigma=sigma, ttm=T, exercise=exercise, K=None)
Merton = Merton_pricer(S0=S0, K=None, ttm=T, r=r, q = q, sigma=0.15, lambd=0.5, meanJ=-0.1, stdJ=0.1, exercise=exercise)
Kou = Kou_pricer(S0=S0, K=None, ttm=T, r=r, sigma=0.15, lambd=0.5, p=0.6, eta1=12, eta2=5, exercise=exercise)
VG = VG_pricer(S0, K=None, ttm=T, r=r, q=q, sigma=0.15, theta=-0.2, nu=0.3, exercise=exercise)

call_th_prices = pd.DataFrame(columns=['Strike','Price', 'BlackScholes', 'Merton', 'Kou', 'VarianceGamma'])
call_th_prices['Strike'] = call_strikes
call_th_prices['Price'] = call_prices

for i, K in enumerate(call_strikes):
    bs = BS.closed_formula_call(K)
    mert = Merton.closed_formula_call(K)
    kou = Kou.closed_formula_call(K)
    vg = VG.closed_formula_call(K)
    call_th_prices.iloc[i, 2:] = [bs, mert, kou, vg]

print(f'Theoretical call options prices:')
print(call_th_prices) #print(call_th_prices.tail(4))

Theoretical call options prices:
   Strike   Price BlackScholes      Merton         Kou VarianceGamma
0  1975.0  81.650   197.684435  104.189464  111.906776    118.316825
1  2275.0   7.750   106.588776   27.565447   34.917953     51.214213
2  2375.0   2.670    85.854828   16.474381   23.141042     38.989506
3  2400.0   2.080    81.279434    14.41281    20.87574     36.445649
4  2425.0   1.270    76.928093   12.585214   18.835063     34.078037
5  2850.0   0.550    29.235022    0.983529    3.622568      11.43898
6  2950.0   0.525    23.124868    0.512573     2.55813      8.974919
7  3000.0   0.500    20.552158    0.368106    2.162431      7.965818


Same for put prices.

In [5]:
put_th_prices = pd.DataFrame(columns=['Strike','Price', 'BlackScholes', 'Merton', 'Kou', 'VarianceGamma'])
put_th_prices['Strike'] = put_strikes
put_th_prices['Price'] = put_prices

for i, K in enumerate(put_strikes):
    bs = BS.closed_formula_put(K)
    mert = Merton.closed_formula_put(K)
    kou = Kou.closed_formula_put(K)
    vg = VG.closed_formula_put(K)
    put_th_prices.iloc[i, 2:] = [bs, mert, kou, vg]

print(f'Theoretical put options prices:')
print(put_th_prices)

Theoretical put options prices:
   Strike    Price BlackScholes     Merton        Kou VarianceGamma
0  1100.0   17.150     5.269774   0.434635    3.34332       0.00594
1  1175.0   22.350     9.375618   0.966768   4.836583      0.029543
2  1300.0   34.060    21.063818   3.128712   8.643503      0.311151
3  1525.0   67.645    63.628095  17.266758   24.48356      8.180387
4  1550.0   72.495    70.413694  20.312392   27.58051     10.856852
5  1650.0   96.245   102.060063  37.049261  44.365303     28.550877
6  1825.0  153.710   175.061769  89.067281  96.455702     91.380433


## Implied volatility
The function belows implements $3$ methods to compute implied volatility: [Newton](https://en.wikipedia.org/wiki/Newton%27s_method) method, the [Bisection](https://en.wikipedia.org/wiki/Bisection_method) method and a more advanced one, named [Brent](https://en.wikipedia.org/wiki/Brent%27s_method) method. Apart from the initial guess, there is no substantial difference in the final result between **Newton** and **bisection** methods (*fsolve*). The **Implied Volatility** is that value $\sigma$ that must be inserted into the Black-Scholes (BS) formula in order to retrieve the option price quoted in the market:
    $$ BS(S, K, T, r, \sigma) = P,  $$
where $S$ is the underlying spot price, $K$ is the strike, $T$ time to maturity, $r$ risk-free interest rate and $P$ the option price quoted in the market. All these quantities are **observable**.
   

In [6]:
def implied_volatility(price, S, strike, t, rate, q, type_o, method='fsolve', disp=True ):
    """ Returns Implied volatility
        methods:  fsolve (default) or brent
    """

    def obj_fun(vol):
        return BS.BlackScholes(type_o=type_o, S0=S, K=strike, ttm=t, r=rate, q=q, sigma=vol) - price

    def vega(vol):
        return BS.vega(S, strike, rate, q, vol, t)

    if method =='fsolve':
        X0 = [0.01, 0.2, 0.35, 7]        #initial guess points for imp.vol.
        for x_0 in X0:
            x, _, solved, _ = scpo.fsolve(obj_fun, x_0, full_output=True, xtol=1e-8)
            if solved == 1:
                return x[0]

    if disp:
        return -1

The following code snippet computes the implied volatility of **call** and **put** options market prices.

In [7]:
IV_market_c = []
for i in range(len(call_prices)):
    IV_market_c.append(implied_volatility(call_prices[i], S=S0, strike=call_strikes[i], t = T, rate=0.02, q = q, type_o='call', method='fsolve'))

print(f'Implied volatilities of market prices (calls):\nS0 = {S0}')
for a,b in zip(call_strikes.tail(6), IV_market_c[-6:]):
    print(f'K = {a}, IV = {round(b, 4)}')

IV_market_p = []
for i in range(len(put_prices)):
    IV_market_p.append(implied_volatility(put_prices[i], S=S0, strike=put_strikes[i], t = T, rate=0.01, q = q, type_o='put', method='fsolve'))

print(f'Implied volatilities of market prices (puts):\nS0 = {S0}')
for a,b in zip(put_strikes.head(6), IV_market_p[:6]):
    print(f'K = {a}, IV = {round(b, 4)}')

Implied volatilities of market prices (calls):
S0 = 1859.48
K = 2375.0, IV = 0.1179
K = 2400.0, IV = 0.1175
K = 2425.0, IV = 0.1132
K = 2850.0, IV = 0.1575
K = 2950.0, IV = 0.1682
K = 3000.0, IV = 0.1728
Implied volatilities of market prices (puts):
S0 = 1859.48
K = 1100.0, IV = 0.3696
K = 1175.0, IV = 0.3545
K = 1300.0, IV = 0.3311
K = 1525.0, IV = 0.2915
K = 1550.0, IV = 0.2869
K = 1650.0, IV = 0.271


##  Weighted Calibration (call options)
Let's step now into the calibration of model parameters.
If we define $\Theta$ the set of parameters, the goal is to find the optimal parameters $\Theta^*$ that minimize the following objective function:
$$ \sum_{i=1}^{N} w_i \biggl( P_i - f(K_i|\Theta) \biggr)^2 $$
where $w_i$ are weights, usually defined as
$$ w_i = \frac{1}{\text {spread}_i },$$ $P_i$ are the market prices and $f$ is the pricing function. In our case $f$ is given by **Merton** Jump Diffusion model, **Kou** Jump Diffusion model, or **Variance Gamma** process. To perform this optimization problem, many numerical methods can be used. In particular, we test two functions of `scipy.optimize`:
1. **curve_fit**, a least-squares curve fitting method which works with bounds. The default algorithm is [Trust Region Reflective (trf)](https://en.wikipedia.org/wiki/Trust_region). The [Levemberg-Marquadt](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm) has been tried as well, to test the optimization problem without setting boundaries, but the results don't make any sense.
2. **Least-Squares**, a constrained minimization problem which uses Trust region reflective method by default. This method is the most indicated to solve the non-linear least squares optimization problem of our purpose.
All the optimizations are carried out by initializing a starting point as the array $x_0 = [params]$ and setting feasible bounds.


In [8]:
call_spreads = calls['C_Spread']
put_spreads = puts['P_Spread']
c_weights = 1/ call_spreads**2
p_weights = 1 / put_spreads**2

### Black and Scholes model
The only unknown parameter to calibrate in Black and Scholes model is the **implied volatility**, $\sigma$. Thus, we minimize the difference between the computed theoretical prices and the market prices of call options.

In [9]:
x0 = 0.5
bounds = [1e-5, 2]

def f_BlackScholes_puts(x, sigm):
    BS = BS_Pricer(S0=S0, K = x, ttm=T, r=r, q=0, sigma=sigm, exercise=exercise)
    return BS.closed_formula_put(x)

res1_puts = scpo.curve_fit(f_BlackScholes_puts, put_strikes, put_prices, p0 = x0, bounds=bounds, sigma=p_weights)
sigw_p = round(res1_puts[0][0],4)

In [10]:
def cost_function(x, strikes, mkt_prices):
    sigma = x
    BS = BS_Pricer(S0=S0, K = None, ttm=T, r=r, q=0, sigma=sigma, exercise=exercise)
    sq_err = np.sum( p_weights* (BS.closed_formula_put(strikes) - mkt_prices)**2)
    return sq_err

result_p = scpo.least_squares(cost_function, x0, args=(put_strikes, put_prices), bounds=bounds, method = 'trf', verbose=1)
opt_sigma_p = result_p.x[0]

`ftol` termination condition is satisfied.
Function evaluations 19, initial cost 3.7084e+07, final cost 1.0993e+04, first-order optimality 1.93e+00.


In [11]:
print('METHOD 1: CURVE_FIT (trf)')
print(f'> Calibrated Volatility from Calls [σ] = {sigw_p} \t {round(sigw_p*100,2)}%')
print('METHOD 2: LEAST-SQUARES (trf)')
print(f'> Calibrated Volatility from Calls [σ] = {opt_sigma_p} \t {round(opt_sigma_p*100,2)}%')

METHOD 1: CURVE_FIT (trf)
> Calibrated Volatility from Calls [σ] = 0.2839 	 28.39%
METHOD 2: LEAST-SQUARES (trf)
> Calibrated Volatility from Calls [σ] = 0.30030215515282876 	 30.03%


### Merton Jump Diffusion
The Merton Jump diffusion ones are the volatility $\sigma$, the Poisson rate of jumps $\lambda$, the mean rate of jump intensity $m$ and its variance rate $v$, assuming that the intensity of jumps follows a *Normal distribution*.


In [12]:
x0 = [0.15,  0.2, -0.05,  0.01]  # initial guess: [σ, λ, m, v]
bounds = ( [1e-3, 1e-2, -5, 1e-5], [2, 5, 5, 2] )

In [13]:
def f_Mert(x, sigma, lambd, meanJ, stdJ):
    Mert = Merton_pricer(S0=S0, K=x, ttm=T, r=r, q=0, sigma=sigma, lambd=lambd, meanJ=meanJ, stdJ=stdJ, exercise=exercise)
    return Mert.closed_formula_put(x)

start1=time.time()
mert1 = scpo.curve_fit(f_Mert, put_strikes, put_prices, p0=x0, bounds=bounds, sigma=put_spreads)
end1=time.time()

mert_params1 = [round(p,4) for p in mert1[0][:4]]

##### Method 2. Least-squares

In [14]:
x0 = [0.1,  0.5, 0.3,  0.1]      # initial guess: [σ, λ, m, v]
bounds = ( [1e-3, 1e-2, -10, 0], [2, np.inf, 10, 5] )

def cost_function(x, strikes, mkt_prices):
    sigma, lambd, meanJ, stdJ = x
    M = Merton_pricer(S0, None, T, r, q, sigma, lambd, meanJ, stdJ, exercise)
    sq_err = np.sum( p_weights*(M.closed_formula_put(strikes) - mkt_prices)**2)
    return sq_err

start2 = time.time()
mert2 = scpo.least_squares(cost_function, x0, args=(put_strikes, put_prices), bounds=bounds, method = 'trf', verbose=2)
end2 = time.time()

mert_params2 = [round(p,4) for p in mert2.x[:4]]

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.9602e+05                                    3.60e+07    
       1              2         4.7688e+05      1.91e+04       5.18e-01       1.46e+08    
       2              4         2.0280e+05      2.74e+05       1.04e-01       2.13e+07    
       3              6         1.5094e+05      5.19e+04       4.44e-02       1.03e+07    
       4              7         1.5071e+05      2.34e+02       4.74e-02       1.40e+07    
       5              8         1.3195e+05      1.88e+04       1.06e-02       8.00e+06    
       6              9         1.1505e+05      1.69e+04       1.91e-02       3.57e+06    
       7             10         1.0242e+05      1.26e+04       1.73e-02       2.50e+06    
       8             11         9.1286e+04      1.11e+04       2.47e-02       1.22e+07    
       9             12         8.7319e+04      3.97e+03       5.13e-02       8.07e+06    

In [15]:
print('WEIGHTED OPT: CURVE_FIT (trf)')
print(f'> Calibrated Volatlity [σ] = {round(mert1[0][0],4)} \t {round(mert1[0][0]*100,2)}%')
print('> Calibrated Jump intensity [λ] = ', round(mert1[0][1],2))
print('> Calibrated Jump Mean = ', round(mert1[0][2],2))
print('> Calibrated Jump St. dev.  = ', round(mert1[0][3],5))
print(f'ELAPSED TIME: {end1-start1} sec')

print('\nMETHOD 1: LEAST SQUARES (trf)')
print(f'> Calibrated Volatlity [σ] = {mert_params2[0]} \t {round(mert_params2[0]*100,2)}%')
print('> Calibrated Jump intensity [λ] = ', round(mert_params2[1],2))
print('> Calibrated Jump Mean = ', round(mert_params2[2],3))
print('> Calibrated Jump St. dev.  = ', round(mert_params2[3],3))
print(f'TIME ELAPSED:  {round(end2-start2,2)} sec')

WEIGHTED OPT: CURVE_FIT (trf)
> Calibrated Volatlity [σ] = 0.1956 	 19.56%
> Calibrated Jump intensity [λ] =  0.26
> Calibrated Jump Mean =  -0.33
> Calibrated Jump St. dev.  =  0.35874
ELAPSED TIME: 2.614861488342285 sec

METHOD 1: LEAST SQUARES (trf)
> Calibrated Volatlity [σ] = 0.1267 	 12.67%
> Calibrated Jump intensity [λ] =  0.95
> Calibrated Jump Mean =  -0.118
> Calibrated Jump St. dev.  =  0.269
TIME ELAPSED:  45.74 sec


In [16]:
print(mert_params1)
print(mert_params2)

[0.1956, 0.2574, -0.3327, 0.3587]
[0.1267, 0.951, -0.1182, 0.2691]


### Kou Jump Diffusion


In [17]:
x0 = [0.2, 0.8, 0.4, 5, 5] # initial guess: [σ, λ, p, η_1, η_2]
bounds = ( [1e-2, 1e-1, 0, 0.5, 0.5], [0.5, 4, 1, 15, 20] )

##### Method 1. TRF (Bounds)

In [18]:
# def f_Kou(x, sigma, lambd, p, eta1, eta2):
#     KouJD = Kou_pricer(S0=S0, K=x, ttm=T, r=r, sigma=sigma, lambd=lambd, p=p, eta1=eta1, eta2=eta2, exercise=exercise)
#     return KouJD.closed_formula_put(x)
#
# start1 = time.time()
# kou1 = scpo.curve_fit(f_Kou, put_strikes, put_prices, p0=x0, bounds=bounds, sigma=p_weights)
# end1 = time.time()
#
# kou_params1 = [round(p,4) for p in kou1[0][:5]]

KeyboardInterrupt: 

##### Method 2. LEAST SQUARES (With Bounds)

In [19]:
# Define the objective function
def cost_function(x, strikes, mkt_prices):
    sigm, lamb, p, eta1, eta2 = x
    KOU = Kou_pricer(S0=S0, K=strikes, ttm=T, r=r, sigma=sigm, lambd=lamb, p=p, eta1=eta1, eta2=eta2, exercise=exercise)
    sq_err = np.sum(p_weights*(KOU.closed_formula_put(strikes) - mkt_prices)**2)
    return sq_err

start2=time.time()
kou2 = scpo.least_squares(cost_function, x0, args=(put_strikes, put_prices),  method='trf', bounds=bounds, verbose=2)
end2=time.time()

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.7238e+03                                    2.28e+04    
       1              3         1.2425e+03      4.81e+02       7.45e-02       1.06e+04    
       2              4         9.2296e+02      3.19e+02       3.67e-02       2.79e+04    
       3              5         9.1114e+02      1.18e+01       7.23e-02       1.20e+04    
       4              6         6.8492e+02      2.26e+02       7.65e-03       7.78e+03    
       5              7         4.6687e+02      2.18e+02       1.62e-02       5.70e+03    
       6              8         3.3347e+02      1.33e+02       2.94e-02       4.71e+03    
       7              9         1.5545e+02      1.78e+02       6.28e-02       2.19e+03    
       8             10         2.1786e+01      1.34e+02       1.45e-01       4.16e+02    
       9             11         2.7212e+00      1.91e+01       5.27e-01       7.01e+01    

In [36]:
kou_params2 = [round(p,4) for p in kou2.x[:5]]

In [37]:
# print('WEIGHTED OPT: CURVE_FIT (trf)')
# print(f'> Calibrated Volatlity [σ] = {kou_params1[0]} \t {kou_params1[0] * 100}%')
# print('> Calibrated Jump intensity [λ] = ', kou_params1[1])
# print(f'> Calibrated Upward Jump probability [p] = {kou_params1[2]}, [q] = {round(1 - kou_params1[2], 2)}')
# print('> Calibrated Rate of Exp. 1  [η_1] = ', kou_params1[3])
# print('> Calibrated Rate of Exp. 2  [η_2] = ', kou_params1[4])
# print(f'TIME ELAPSED: {end1-start1} sec')

In [38]:
print('METHOD 2: Least-squares')
print(f'> Calibrated Volatlity [σ] = {round(kou_params2[0],4)} \t {round(kou_params2[0]*100,2)}%')
print('> Calibrated Jump intensity [λ] = ', round(kou_params2[1],2))
print(f'> Calibrated Upward Jump probability [p] = {round(kou_params2[2],2)}, [q] = {round(1-kou_params2[2],2)}')
print('> Calibrated Rate of Exp. 1  [η_1] = ', round(kou_params2[3],2))
print('> Calibrated Rate of Exp. 2  [η_2] = ', round(kou_params2[4],2))
print(f'TIME ELAPSED:  {round(end2-start2,2)} sec')

METHOD 2: Least-squares
> Calibrated Volatlity [σ] = 0.1278 	 12.78%
> Calibrated Jump intensity [λ] =  0.95
> Calibrated Upward Jump probability [p] = 0.35, [q] = 0.65
> Calibrated Rate of Exp. 1  [η_1] =  5.72
> Calibrated Rate of Exp. 2  [η_2] =  4.23
TIME ELAPSED:  184.89 sec


In [39]:
#print(kou_params1)
print(kou_params2)

[0.1278, 0.948, 0.3549, 5.7153, 4.232]


### Variance Gamma


In [40]:
x0 = [0.2, -0.3, 0.1]   # initial guess: [σ, θ, v]
bounds = ( [1e-3, -5, 0], [3, 5, 10] )

##### Method 1. CURVE FIT (Bounds)

In [41]:
def f_VG(strikes, sigmax, thetax, nux):
    VGamma = VG_pricer(S0=S0, K=None, ttm=T, r=r, q=0, sigma=sigmax, theta=thetax, nu=nux, exercise='put')
    vg_prices = []
    for k in strikes:
        vg_prices.append(VGamma.closed_formula_put(k))
    return vg_prices

start1 = time.time()
vg1 = scpo.curve_fit(f_VG, put_strikes, put_prices, p0=x0, bounds=bounds, sigma=p_weights)
end1 = time.time()

vg_params1 = [round(p,4) for p in vg1[0][:3]]

##### Method 2. LEAST-SQUARES (Trust Region Reflective, Bounds)

In [42]:
def cost_function(x, strikes, mkt_prices):
    sigma, theta, nu = x
    VG = VG_pricer(S0, None, T, r, q, sigma, theta, nu, exercise)
    prices = []
    for k in strikes:
        prices.append(VG.closed_formula_put(k))
    sq_err = np.sum(p_weights*(prices - mkt_prices)**2)
    return sq_err

start2=time.time()
vg2 = scpo.least_squares(cost_function, x0, args=(put_strikes, put_prices),  method='trf', bounds=bounds, verbose=2, loss='soft_l1')
end2=time.time()

vg_params2 = [round(p,4) for p in vg2.x[:3]]

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         9.1498e+02                                    2.79e+04    
       1              3         2.9987e+02      6.15e+02       8.27e-02       1.15e+04    
       2              5         2.5024e+02      4.96e+01       3.84e-02       3.12e+03    
       3              6         2.2516e+02      2.51e+01       4.14e-02       1.56e+03    
       4              8         2.2224e+02      2.92e+00       1.17e-02       2.02e+03    
       5              9         2.1724e+02      5.00e+00       1.23e-02       1.78e+03    
       6             10         2.0898e+02      8.26e+00       2.60e-02       1.49e+03    
       7             12         2.0690e+02      2.09e+00       1.15e-02       1.91e+03    
       8             13         2.0200e+02      4.89e+00       1.19e-02       1.71e+03    
       9             14         1.9404e+02      7.96e+00       2.57e-02       1.47e+03    

In [49]:
print('WEIGHTED OPT: CURVE_FIT (trf)')
print(f'> Calibrated Volatlity [σ] = {vg_params1[0]}, \t {round(vg_params1[0]*100,2)}%')
print('> Calibrated mean rate gamma process [θ] = ', vg_params1[1])
print('> Calibrated variance rate gamma process [v]= ', vg_params1[2])
print(f'TIME ELAPSED:  {round(end1-start1,2)} sec')

print('METHOD 1: LEAST-SQUARES (trf)')
print(f'> Calibrated Volatlity [σ] = {vg_params2[0]}, \t {round(vg_params2[0]*100,2)}%')
print('> Calibrated mean rate gamma process [θ] = ', vg_params2[1])
print('> Calibrated variance rate gamma process [v]= ', vg_params2[2])
print(f'TIME ELAPSED:  {round(end2-start2,2)} sec')

WEIGHTED OPT: CURVE_FIT (trf)
> Calibrated Volatlity [σ] = 0.3405, 	 34.05%
> Calibrated mean rate gamma process [θ] =  0.0327
> Calibrated variance rate gamma process [v]=  2.1571
TIME ELAPSED:  22.51 sec
METHOD 1: LEAST-SQUARES (trf)
> Calibrated Volatlity [σ] = 0.2442, 	 24.42%
> Calibrated mean rate gamma process [θ] =  0.1983
> Calibrated variance rate gamma process [v]=  0.9451
TIME ELAPSED:  189.91 sec


In [50]:
print(vg_params1)
print(vg_params2)

[0.3405, 0.0327, 2.1571]
[0.2442, 0.1983, 0.9451]


### Reprice of options using calibrated parameters
The following code snippet aims to find the best parameters for each model, minimizing the difference between the monte carlo prices and the market prices of **call** and **put** options.

In [12]:
days = 252
paths = 5000

put_calib_prices = pd.DataFrame({
    'STRIKE': puts['STRIKE'],    # array of K for call options
    'MKT_BID': puts['P_BID'],
    'MKT_MID': puts['P_Midpoint'],
    'MKT_ASK': puts['P_ASK'],

})
print(f'MARKET PRICES. STARTING DATE = 20-01-2016. EXPIRY = 1YEAR. \n\n{put_calib_prices}')

MARKET PRICES. STARTING DATE = 20-01-2016. EXPIRY = 1YEAR. 

   STRIKE  MKT_BID  MKT_MID  MKT_ASK
0  1100.0    16.10   17.150    18.20
1  1175.0    21.30   22.350    23.40
2  1300.0    32.91   34.060    35.21
3  1525.0    66.29   67.645    69.00
4  1550.0    71.10   72.495    73.89
5  1650.0    94.79   96.245    97.70
6  1825.0   152.01  153.710   155.41


#### Merton Jump Diffusion model

In [13]:
mert_params2 = [0.1267, 0.951, -0.1182, 0.2691]
sigma, lambd, meanJ, stdJ = mert_params2
MertonCAL = Merton_pricer(S0, None, T, r, q, sigma, lambd, meanJ, stdJ, exercise)
SMerton_CAL = MertonCAL.MertonPath(days, paths)

avg_payoffs = []
for K in put_strikes:
    payoffs = []        # stores here the payoff for each path, for a specific couple K1-K2
    for St in SMerton_CAL[-1]:
        payoffs.append(MertonCAL.payoff_put(K, St))
    avg_payoffs.append(np.mean(payoffs))

merton_mc_prices = np.zeros(len(put_calib_prices))
merton_cf_prices = np.zeros(len(put_calib_prices))

for index in range(len(put_calib_prices)):
    merton_mc_prices[index] = np.exp(-r*T)* avg_payoffs[index]
    merton_cf_prices[index] = MertonCAL.closed_formula_put(put_strikes[index])

put_calib_prices['MERTON MC'] = merton_mc_prices
put_calib_prices['MERTON CF'] = merton_cf_prices

print(put_calib_prices)

   STRIKE  MKT_BID  MKT_MID  MKT_ASK   MERTON MC   MERTON CF
0  1100.0    16.10   17.150    18.20    6.625912   13.911466
1  1175.0    21.30   22.350    23.40    9.974092   19.640835
2  1300.0    32.91   34.060    35.21   18.593132   32.875558
3  1525.0    66.29   67.645    69.00   48.401113   71.010238
4  1550.0    71.10   72.495    73.89   53.152339   76.568466
5  1650.0    94.79   96.245    97.70   75.580047  101.922191
6  1825.0   152.01  153.710   155.41  129.927666  160.957360


In [14]:
kou_params2 = [0.1278, 0.948, 0.3549, 5.7153, 4.232]
sigma, lambd, p, eta1, eta2 = kou_params2
KouCAL = Kou_pricer(S0, None, T, r, sigma, lambd, p, eta1, eta2, exercise)
SKou_CAL = KouCAL.KouPath(days, paths)

avg_payoffs = []
for K in put_strikes:
    payoffs = []        # stores here the payoff for each path, for a specific couple K1-K2
    for St in SKou_CAL[-1]:
        payoffs.append(KouCAL.payoff_put(K, St))
    avg_payoffs.append(np.mean(payoffs))

kou_mc_prices = np.zeros(len(put_calib_prices))
kou_cf_prices = np.zeros(len(put_calib_prices))

for index in range(len(put_calib_prices)):
    kou_mc_prices[index] = np.exp(-r*T)* avg_payoffs[index]
    kou_cf_prices[index] = KouCAL.closed_formula_put(put_strikes[index])

put_calib_prices['KOU MC'] = kou_mc_prices
put_calib_prices['KOU CF'] = kou_cf_prices

print(put_calib_prices)

   STRIKE  MKT_BID  MKT_MID  MKT_ASK   MERTON MC   MERTON CF      KOU MC   
0  1100.0    16.10   17.150    18.20    6.625912   13.911466   15.421320  \
1  1175.0    21.30   22.350    23.40    9.974092   19.640835   20.702902   
2  1300.0    32.91   34.060    35.21   18.593132   32.875558   32.362210   
3  1525.0    66.29   67.645    69.00   48.401113   71.010238   64.604532   
4  1550.0    71.10   72.495    73.89   53.152339   76.568466   69.344219   
5  1650.0    94.79   96.245    97.70   75.580047  101.922191   91.563645   
6  1825.0   152.01  153.710   155.41  129.927666  160.957360  148.256286   

       KOU CF  
0   16.768477  
1   22.243934  
2   34.243199  
3   67.757558  
4   72.720483  
5   95.940138  
6  153.659904  


In [15]:
vg_params2 = [0.2442, 0.1983, 0.9451]
sigma, theta, nu = vg_params2
VGCAL = VG_pricer(S0, None, T, r, q, sigma, theta, nu, exercise)
SVarGamma_CAL = VGCAL.VarianceGammaPath1(days, paths)

avg_payoffs = []
for K in put_strikes:
    payoffs = []        # stores here the payoff for each path, for a specific couple K1-K2
    for St in SVarGamma_CAL[-1]:
        payoffs.append(KouCAL.payoff_put(K, St))
    avg_payoffs.append(np.mean(payoffs))

vg_mc_prices = np.zeros(len(put_calib_prices))
vg_cf_prices = np.zeros(len(put_calib_prices))

for index in range(len(put_calib_prices)):
    vg_mc_prices[index] = np.exp(-r*T)* avg_payoffs[index]
    vg_cf_prices[index] = VGCAL.closed_formula_put(put_strikes[index])

put_calib_prices['VG MC'] = vg_mc_prices
put_calib_prices['VG CF'] = vg_cf_prices

print(put_calib_prices)

   STRIKE  MKT_BID  MKT_MID  MKT_ASK   MERTON MC   MERTON CF      KOU MC   
0  1100.0    16.10   17.150    18.20    6.625912   13.911466   15.421320  \
1  1175.0    21.30   22.350    23.40    9.974092   19.640835   20.702902   
2  1300.0    32.91   34.060    35.21   18.593132   32.875558   32.362210   
3  1525.0    66.29   67.645    69.00   48.401113   71.010238   64.604532   
4  1550.0    71.10   72.495    73.89   53.152339   76.568466   69.344219   
5  1650.0    94.79   96.245    97.70   75.580047  101.922191   91.563645   
6  1825.0   152.01  153.710   155.41  129.927666  160.957360  148.256286   

       KOU CF     VG MC       VG CF  
0   16.768477  0.018971   16.208267  
1   22.243934  0.033528   21.734253  
2   34.243199  0.078977   34.060445  
3   67.757558  0.349748   69.145741  
4   72.720483  0.433903   74.309188  
5   95.940138  0.918902   97.993856  
6  153.659904  2.546824  152.939533  
